In [1]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath('../src'))

from data_load import load_data
from data_load import describe_dataset
from eda import convert_to_datetime
from eda import clean_features_data
from eda import calculate_and_save_statistics


In [2]:
# Load datasets
sales_df = load_data(r'C:\Users\007ri\Product_EDA/data/sales data-set.csv')
stores_df = load_data(r'C:\Users\007ri\Product_EDA/data/stores data-set.csv')
features_df = load_data(r'C:\Users\007ri\Product_EDA/data/Features data set.csv')

# display 1st few rows
sales_df.head(), stores_df.head(), features_df.head()


Successfully loaded data from C:\Users\007ri\Product_EDA/data/sales data-set.csv
Successfully loaded data from C:\Users\007ri\Product_EDA/data/stores data-set.csv
Successfully loaded data from C:\Users\007ri\Product_EDA/data/Features data set.csv


(   Store  Dept        Date  Weekly_Sales  IsHoliday
 0      1     1  05/02/2010      24924.50      False
 1      1     1  12/02/2010      46039.49       True
 2      1     1  19/02/2010      41595.55      False
 3      1     1  26/02/2010      19403.54      False
 4      1     1  05/03/2010      21827.90      False,
    Store Type    Size
 0      1    A  151315
 1      2    A  202307
 2      3    B   37392
 3      4    A  205863
 4      5    B   34875,
    Store        Date  Temperature  Fuel_Price  MarkDown1  MarkDown2  \
 0      1  05/02/2010        42.31       2.572        NaN        NaN   
 1      1  12/02/2010        38.51       2.548        NaN        NaN   
 2      1  19/02/2010        39.93       2.514        NaN        NaN   
 3      1  26/02/2010        46.63       2.561        NaN        NaN   
 4      1  05/03/2010        46.50       2.625        NaN        NaN   
 
    MarkDown3  MarkDown4  MarkDown5         CPI  Unemployment  IsHoliday  
 0        NaN        NaN        N

In [3]:
 # Save dataset info to a text file
with open(r"C:\Users\007ri\Product_EDA/reports/datasets_info.txt", "w", encoding="utf-8") as file:
    describe_dataset(sales_df, "Sales Data", file)
    describe_dataset(stores_df, "Stores Data", file)
    describe_dataset(features_df, "Features Data", file)

print("Dataset info saved to 'dataset_summary.txt'")

Dataset info saved to 'dataset_summary.txt'


In [3]:
# Convert 'Date' columns to datetime format
sales_df["Date"] = convert_to_datetime(sales_df)
features_df["Date"] = convert_to_datetime(features_df)

# Fill Markdown columns missing values with 0 and CPI & Unemployment missing values using forward fill
features_df = clean_features_data(features_df)

# Check for missing values
print(sales_df.isnull().sum())
print(features_df.isnull().sum())
print(stores_df.isnull().sum())


Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64
Store           0
Date            0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
IsHoliday       0
dtype: int64
Store    0
Type     0
Size     0
dtype: int64


In [4]:
# Calculate mean, median, and standard deviation for numeric columns of a DataFrame and save it to a text file
with open(r"C:\Users\007ri\Product_EDA/reports/summary_statistics.txt", "w", encoding="utf-8") as file:

    calculate_and_save_statistics(sales_df, "Sales Data", file)
    calculate_and_save_statistics(stores_df, "Stores Data", file)
    calculate_and_save_statistics(features_df, "Features Data", file)

print("Statistics summary saved to file")


Statistics summary saved to file


In [9]:
# Merge sales with store details
merged_df = sales_df.merge(stores_df, on="Store", how="left")

# Merge with features dataset
merged_df = merged_df.merge(features_df, on=["Store", "Date","IsHoliday"], how="left")

# Display the merged data structure
print(merged_df.info())

# Merged dataset is written into a file for generating Power BI reports
merged_df.to_csv(r"C:\Users\007ri\Product_EDA/data/processed_merged_dataset.csv", index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Store         421570 non-null  int64         
 1   Dept          421570 non-null  int64         
 2   Date          421570 non-null  datetime64[ns]
 3   Weekly_Sales  421570 non-null  float64       
 4   IsHoliday     421570 non-null  bool          
 5   Type          421570 non-null  object        
 6   Size          421570 non-null  int64         
 7   Temperature   421570 non-null  float64       
 8   Fuel_Price    421570 non-null  float64       
 9   MarkDown1     421570 non-null  float64       
 10  MarkDown2     421570 non-null  float64       
 11  MarkDown3     421570 non-null  float64       
 12  MarkDown4     421570 non-null  float64       
 13  MarkDown5     421570 non-null  float64       
 14  CPI           421570 non-null  float64       
 15  Unemployment  421

In [12]:
# Summary statistics
print("Summary Statistics:\n", merged_df.describe())

# Sales Summary by Store Type
print("\nSales Summary by Store Type:\n", merged_df.groupby("Type")["Weekly_Sales"].agg(["mean", "median", "std"]))

# Sales Summary by Holiday
print("\nSales Summary by Holiday:\n", merged_df.groupby("IsHoliday")["Weekly_Sales"].agg(["mean", "median", "std"]))

# Sales Summary by Store number
print("\nSales Summary by Store:\n", merged_df.groupby("Store")["Weekly_Sales"].agg(["mean", "median", "std"]))


Summary Statistics:
                Store           Dept                           Date  \
count  421570.000000  421570.000000                         421570   
mean       22.200546      44.260317  2011-06-18 08:30:31.963375104   
min         1.000000       1.000000            2010-02-05 00:00:00   
25%        11.000000      18.000000            2010-10-08 00:00:00   
50%        22.000000      37.000000            2011-06-17 00:00:00   
75%        33.000000      74.000000            2012-02-24 00:00:00   
max        45.000000      99.000000            2012-10-26 00:00:00   
std        12.785297      30.492054                            NaN   

        Weekly_Sales           Size    Temperature     Fuel_Price  \
count  421570.000000  421570.000000  421570.000000  421570.000000   
mean    15981.258123  136727.915739      60.090059       3.361027   
min     -4988.940000   34875.000000      -2.060000       2.472000   
25%      2079.650000   93638.000000      46.680000       2.933000   
50%